In [7]:
import pandas as pd
%matplotlib inline
data = pd.read_csv('./data/exam/CM2.csv', na_values='?')
data = data.dropna(axis=0)

C:\Projects\machine-learning\exam


,MPG,N1,N2,N4,N5,C1,C2
MPG,1.000000,-0.719937,-0.776671,-0.850012,0.257135,0.514602,0.499183
N1,-0.719937,1.000000,0.939252,0.857017,-0.266400,-0.366997,-0.449003
N2,-0.776671,0.939252,1.000000,0.922638,-0.273702,-0.349186,-0.560364
N4,-0.850012,0.857017,0.922638,1.000000,-0.128839,-0.366271,-0.538370
N5,0.257135,-0.266400,-0.273702,-0.128839,1.000000,0.157764,0.100735
C1,0.514602,-0.366997,-0.349186,-0.366271,0.157764,1.000000,0.126280
C2,0.499183,-0.449003,-0.560364,-0.538370,0.100735,0.126280,1.000000


In [8]:
from sklearn import preprocessing
def bin_feature(df, feature):
    binarizer = preprocessing.LabelBinarizer()
    nw = binarizer.fit_transform(df[feature])
    classes = binarizer.classes_
    new_names = list(map(lambda cls: feature + " eq " + str(cls), classes))
    if len(binarizer.classes_) == 2:
        new_names = new_names[1:]
    new_data = pd.DataFrame(nw, columns=new_names)
    new_data.index = df.index
    new_data = pd.concat([df, new_data], axis=1)
    new_data = new_data.drop(feature, axis=1)
    return new_data

In [9]:
from sklearn import cross_validation
data = bin_feature(data, 'C1')
data = bin_feature(data, 'C2')
X = data.drop(['MPG'], axis=1)
Y = data['MPG']
x_train, x_test, y_train, y_test = cross_validation.train_test_split(X, Y, test_size=0.2, random_state=11)

In [10]:
from sklearn import ensemble
import math
best_tree = 10
best_score = 1000000000.
for tree_count in range(10, 500, 70):
    model = ensemble.RandomForestRegressor(n_estimators=tree_count, max_depth=8, random_state=11)
    model.fit(x_train, y_train)
    res = model.predict(x_test)
    fail = 0.
    for (r, w) in zip(res, y_test):
        fail += (r - w) * (r - w)     
    fail = math.sqrt(fail)
    if fail < best_score:
        best_tree = tree_count
        best_score = fail

print('Best result: trees = {}, fail = {}'. format(best_tree, best_score))

ValueError: could not convert string to float: '?'

9537 20558
